# AIRPLANE ACCIDENTS IN LATAM 
By HAMM

**Introduction**

Latin America is a group of countries in which tourism represents one of the most important activities of GDP: foreigners who visit its coasts and also our inland cities and in many cases not even the famous and dreaded economic recession has been able to Stop this phenomenon. In fact, the facilities that they both find to come to this region and those that Latin Americans find to leave their respective countries are growing, especially if they opt for air transport.

Nowadays, any young person has traveled by plane, or if they have not done so, they have certainly had the opportunity, something that for other people of past generations was totally unthinkable: air transport was seen decades ago as something reserved for business people or elite families.

On the other hand, many people are afraid of flying and the news of air accidents, although not very common, increases this fear because most of these accidents are fatal. I decided to do this study on the accidents of the last era in Latin America because in the news we often hear accidents in developed countries such as the United States or China, which has never affected its economy, but when it comes to a third world country it is Another story Not only does tourism affect itself, but it also affects the image of the region causing fear in foreigners.

With this work I intend to find both geographical and political relationships that allow us to understand the background of accidents in Latin American countries.

**Data**

*Libraries*

In [1]:
import numpy as np 
import pandas as pd 
import json 
import requests 
import codecs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import datetime

from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('OK Libraries')

OK Libraries


The database that I am going to handle was taken from Wikipedia with the global air accidents of recent years. 

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_aircraft_accidents_and_incidents_resulting_in_at_least_50_fatalities').text

In [3]:
soup = BeautifulSoup(website_url,'lxml')
tables = soup.find_all("table", { "class" : "wikitable" })

In [4]:
def parse(n):
    data = []
    for row in tables[n].find_all("tr"):
        if row and row.find_all('td'):
            data_row = []
            cells = row.find_all('td')
            for cell in cells:
                if cell:
                    data_row.append(cell.get_text().rstrip())            
            data.append(data_row)
    return data

In [5]:
data = pd.DataFrame(parse(2), columns = ['Type','Incident','Aircraft','Location','Phase','Airport','Distance','Date']) 

In [6]:
x = data.Location
for i in range (len(data)) :
    y= data.Location[i]
    z=len(y)
    AA=0
    while y[AA].islower():
        x[i] = y[AA+1:z]
        AA=AA+1
        data.Location=x

This will only filter those found in Latin American countries without taking into account their city of origin, model of airplane or city of origin. 

In [7]:
loc=data['Location']

In [8]:
latam=['Chile','Uruguay','Puerto Rico','Colombia','Mexico','Cuba','Argentina','Ecuador','Brazil','Venezuela','Honduras','Nicaragua','Guatemala','Peru','Bolivia']

In [9]:
mask = np.logical_or.reduce([loc.str.contains(i, regex=False, case=False) for i in latam])

In [10]:
Air_Latam=data[mask]

In [11]:
Air_Latam=Air_Latam.drop([95],axis=0)

In [12]:
Air_Latam.head()

,Type,Incident,Aircraft,Location,Phase,Airport,Distance,Date
34,COM,TAM Airlines Flight 3054,Airbus A320-233,"São Paulo, Brazil",LDG[64],CGH,,2007-07-17
55,COM,Mexicana Flight 940,Boeing 727-264,"Sierra Madre Oriental Mts., Mexico",ENR[89],,,1986-03-31
59,COM,West Caribbean Airways Flight 708,McDonnell Douglas MD-82,"Machiques, Venezuela",ENR[93],,,2005-08-16
62,COM,American Airlines Flight 965,Boeing 757-223,"Buga, Colombia",APR[96],CLO,c. 48 km(30 mi),1995-12-20
72,COM,Viasa Flight 742,McDonnell Douglas DC-9-32,"Maracaibo, Venezuela",ICL[107],MAR,0.85 km(0.5 mi),1969-03-16


In [13]:
Air_Latam.shape

(67, 8)

Using one of the types of maps learned during the course, the geography of the region will be analyzed for accident patterns.

In [14]:
Air_Latam['Latitude'] = np.nan
Air_Latam['Longitude'] = np.nan

In [15]:
    for i in Air_Latam.index : 
        lok= Air_Latam.Location[i]

        try:
            geo = Nominatim(user_agent="ny_explorer")
            loc = geo.geocode(lok)
            Air_Latam.Latitude[i] = loc.latitude
            Air_Latam.Longitude[i] = loc.longitude
        except:
            x=0

C:\Users\bullt\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\bullt\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
Air_Latam=Air_Latam.dropna()

**Analysis**

First of all we need to see the physical map of LATAM

In [17]:
latitude = 4.3635
longitude = -74.0454
map_AirLatam = folium.Map(location=[latitude, longitude],tiles='Stamen Terrain', zoom_start=3)

In [18]:
map_AirLatam

In [19]:
map_AirLatam.save('MapLATAM.html')

Now we see the accidents location in the terrain map. On the map we find the most important accidents such as the Andes mountain range, the great Chaco, the Argentine pampas, the Patagonia desert and Fire Land.

In [20]:
folium.Marker([-18.32, -69.56], popup='Andes', icon = folium.Icon(color ='green')).add_to(map_AirLatam)
folium.Marker([-35.12, -67.10], popup='Pampa', icon = folium.Icon(color ='green')).add_to(map_AirLatam)
folium.Marker([-48.01, -70.70], popup='Patagonia', icon = folium.Icon(color ='green')).add_to(map_AirLatam)
map_AirLatam.save('MapLATAMmarkers.html')

In [21]:
map_AirLatam

In [22]:
latitude = 4.3635
longitude = -74.0454
map_AirLatam1 = folium.Map(location=[latitude, longitude],tiles='Stamen Terrain', zoom_start=3)

for lat, lng, location in zip(Air_Latam['Latitude'],Air_Latam['Longitude'],Air_Latam['Location']):
    label = '{}'.format(location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        parse_html=False).add_to(map_AirLatam1)  

In [23]:
map_AirLatam1.save('Air_Latam.html')

In [24]:
map_AirLatam1

As we can see on the map, we find that most of the accidents occur due to geographical faults in the region, mainly in the Andes mountain range, since crossing all of South America it is more common for airplanes to travel between the mountains.

You can see that the largest amount is concentrated between Ecuador and Colombia, since in the latter the coordination is divided into 3 making geography much more difficult.

Finally we see that in the other geographical failures there have been accidents.

**conclusion**

To conclude the work we find that geographical failures are the cause of most of the aerial tragedies in Latin America.
Although we all know that the region is not only difficult because of geography but also because of political, social and forced displacement situations.

We found that most of the accidents occurred in the north of his America since there are the high winds, winds that come from the Pacific and the mountains are much more difficult to travel.